In [1]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00


In [3]:
!mkdir diffusionSamples

In [4]:
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, MidiProcessor, SpectrogramDiffusionPipeline
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2024-05-08 03:34:50.914507: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 03:34:50.914599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 03:34:51.030883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
pipe = StableDiffusionPipeline.from_pretrained("riffusion/riffusion-model-v1")
pipe = pipe.to("cuda")

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

safety_checker/model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

safety_checker/pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

unet/diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/conda/lib/python3.10/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:236: FutureWarning: The configuration file of the unet has set the default `sample_size` to smaller than 64 which seems highly unlikely. If your checkpoint is a fine-tuned version of any of the following: 
- CompVis/stable-diffusion-v1-4 
- CompVis/stable-diffusion-v1-3 
- CompVis/stable-diffusion-v1-2 
- CompVis/stable-diffusion-v1-1 
- runwayml/stable-diffusion-v1-5 
- runwayml/stable-diffusion-inpainting 
 you should change 'sample_size' to 64 in the configuration file. Please make sure to upda

In [6]:
from PIL import Image
from torchvision import transforms

In [7]:
pipe.scheduler.set_timesteps(50, device="cuda")
T = pipe.scheduler.timesteps
T = torch.flip(T, dims=(0,))
stepsize = 5

In [15]:
idxs = range(0, 50, stepsize)

In [16]:
prompt = "Soothing slow music"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
uncond_tokens = [""] * 1
text_inputs = pipe.tokenizer(
    prompt,
    padding="max_length",
    max_length=pipe.tokenizer.model_max_length,
    return_tensors="pt",
)
text_input_ids = text_inputs.input_ids
text_embeddings = pipe.text_encoder(text_input_ids.to(device))[0]

bs_embed, seq_len, _ = text_embeddings.shape
text_embeddings = text_embeddings.repeat(1, 1, 1)
text_embeddings = text_embeddings.view(bs_embed * 1, seq_len, -1)

max_length = text_input_ids.shape[-1]
uncond_input = pipe.tokenizer(
    uncond_tokens,
    padding="max_length",
    max_length=max_length,
    truncation=True,
    return_tensors="pt",
)
uncond_embeddings = pipe.text_encoder(uncond_input.input_ids.to(device))[0]

seq_len = uncond_embeddings.shape[1]
uncond_embeddings = uncond_embeddings.repeat(1, 1, 1)
uncond_embeddings = uncond_embeddings.view(1 * 1, seq_len, -1)

text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

In [18]:
latents_list = []

def callback(iter, t, latents):
    # convert latents to image
    with torch.no_grad():
        latents = 1 / 0.18215 * latents
        
        latents_list.append(latents)
        
        latent = torch.cat([latents_list[0]] * 2)
        
        latent_model_input = pipe.scheduler.scale_model_input(latent, T[idxs[0]])
        
        noise_pred = pipe.unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample
        
        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        
        noise_pred = noise_pred_uncond + 7.5 * (noise_pred_text - noise_pred_uncond)
        
        noise_pred = 1 / 0.18215 * noise_pred
    
        noise_image = pipe.vae.decode(noise_pred).sample

        noise_image = (noise_image / 2 + 0.5).clamp(0, 1)

        # we always cast to float32 as this does not cause significant overhead and is compatible with bfloa16
        noise_image = noise_image.cpu().permute(0, 2, 3, 1).float().numpy()

        # convert to PIL Images
        noise_image = pipe.numpy_to_pil(noise_image)
        
        image = pipe.vae.decode(latents).sample

        image = (image / 2 + 0.5).clamp(0, 1)

        # we always cast to float32 as this does not cause significant overhead and is compatible with bfloa16
        image = image.cpu().permute(0, 2, 3, 1).float().numpy()

        # convert to PIL Images
        image = pipe.numpy_to_pil(image)

        # do something with the Images
        for i, img in enumerate(image):
            img.save(f"diffusionSamples/iter_{iter}_img{i}.png")
        
        for i, img in enumerate(noise_image):
            img.save(f"diffusionSamples/noise_pred_iter_{iter}_img{i}.png")

# generate image (note the `callback` and `callback_steps` argument)
#image = model("tree", callback=callback, callback_steps=5)
image = pipe(prompt, callback=callback, callback_steps=5).images[0]

  0%|          | 0/50 [00:00<?, ?it/s]

In [19]:
image.save("soothingSlowMusic.jpg")

In [20]:
image = Image.open("soothingSlowMusic.jpg").convert("RGB")
image = transforms.ToTensor()(image)

images = []
for idx in idxs:
    t = T[idx]
    noise = torch.randn_like(image)
    noisy_image = pipe.scheduler.add_noise(image, noise, t)
    images.append(noisy_image)
    
for i in range(10):
    transforms.ToPILImage()(images[i]).save(f"diffusionSamples/forward_sample_{i}.png")

t = torch.Tensor([60]).to("cuda").to(torch.int64)
noise = torch.randn_like(image)
noisy_image = pipe.scheduler.add_noise(image, noise, t)
transforms.ToPILImage()(noisy_image)
transforms.ToPILImage()(noisy_image).save(f"diffusionSamples/forward_sample_man_3.png")

In [21]:
!zip -r diffusionSampleWithNoise.zip ./diffusionSamples/

  adding: diffusionSamples/ (stored 0%)
  adding: diffusionSamples/noise_pred_iter_20_img0.png (deflated 0%)
  adding: diffusionSamples/iter_5_img0.png (deflated 0%)
  adding: diffusionSamples/forward_sample_8.png (deflated 0%)
  adding: diffusionSamples/iter_25_img0.png (deflated 0%)
  adding: diffusionSamples/iter_45_img0.png (deflated 0%)
  adding: diffusionSamples/forward_sample_4.png (deflated 0%)
  adding: diffusionSamples/iter_40_img0.png (deflated 0%)
  adding: diffusionSamples/forward_sample_3.png (deflated 0%)
  adding: diffusionSamples/forward_sample_2.png (deflated 0%)
  adding: diffusionSamples/noise_pred_iter_35_img0.png (deflated 0%)
  adding: diffusionSamples/noise_pred_iter_15_img0.png (deflated 0%)
  adding: diffusionSamples/forward_sample_1.png (deflated 0%)
  adding: diffusionSamples/forward_sample_5.png (deflated 0%)
  adding: diffusionSamples/noise_pred_iter_5_img0.png (deflated 0%)
  adding: diffusionSamples/iter_20_img0.png (deflated 0%)
  adding: diffusionSampl